In [3]:
import opstrat as op
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
import seaborn as sns

import json
with open('config.json') as config_file:
     data = json.load(config_file)
print(data)

last_price          = data['Current_Price']
annual_drift        = data['Annual_Drift']
annual_volatility   = data['Annual_Volatiltiy']
time_period         = data['Time_Period']
simu                = data['No_of_days']
r                   = data['rate_return']
sigma               = data['sigma']
price               = data['Current_Price']

df = pd.DataFrame()

last_price_list = [last_price]
diff_list       = []
tenor_list      = [0.083333333333333333]
count           = 0
count_list      = []
while count < simu:
    count = count + 1
    difference = last_price*annual_drift*time_period + last_price*annual_volatility*(time_period**(0.5))*norm.ppf(random.random())

    last_price = last_price + difference
    tenor      = ((30-count)/(30*12))
    last_price_list.append(last_price)
    diff_list.append(difference)
    count_list.append(count)
    tenor_list.append(tenor)
last_price_list.pop(-1)
tenor_list.pop(-1)
df['Count'] = count_list
df['Price'] = last_price_list
df['Change_price'] = diff_list
df['tenor'] = tenor_list


datalists = dict()
k,p = int(round(price, -2))+800,int(round(price, -2))-800
  
call_list  = []
put_list   = []
   
for last_price,t in zip(df['Price'],df['tenor']):

    call   = -(last_price*(norm.cdf((math.log(last_price/k)+(r+((sigma**2)/2))*t)/(sigma*(t**(0.5)))))-k*math.exp(-r*t)*(norm.cdf(((math.log(last_price/k)+(r+((sigma**2)/2))*t)/(sigma*(t**(0.5)))-sigma*(t**0.5)))))
    put    = -(p*math.exp(-r*t)*(1-norm.cdf(((math.log(last_price/p)+(r+((sigma**2)/2))*t))/(sigma*(t**(0.5)))-sigma*(t**0.5)))-last_price*(1-norm.cdf((math.log(last_price/p)+(r+((sigma**2)/2))*t)/(sigma*(t**(0.5))))))
    
    call_list.append(call)  
    put_list.append(put)
    datalists['CE_'+str(k)] = call_list  # CP = Call Premium
    datalists['PE_'+str(p)] = put_list   # PP = Put Premium

k,p = int(round(price, -2))+500,int(round(price, -2))-500
    
call_list  = []
put_list   = []
    
for last_price,t in zip(df['Price'],df['tenor']):
    call   = (last_price*(norm.cdf((math.log(last_price/k)+(r+((sigma**2)/2))*t)/(sigma*(t**(0.5)))))-k*math.exp(-r*t)*(norm.cdf(((math.log(last_price/k)+(r+((sigma**2)/2))*t)/(sigma*(t**(0.5)))-sigma*(t**0.5)))))
    put    = (p*math.exp(-r*t)*(1-norm.cdf(((math.log(last_price/p)+(r+((sigma**2)/2))*t))/(sigma*(t**(0.5)))-sigma*(t**0.5)))-last_price*(1-norm.cdf((math.log(last_price/p)+(r+((sigma**2)/2))*t)/(sigma*(t**(0.5))))))
    
    call_list.append(call)  
    put_list.append(put)
    datalists['CE_'+str(k)] = call_list  # CP = Call Premium
    datalists['PE_'+str(p)] = put_list   # PP = Put Premium


df1 = pd.DataFrame.from_dict(datalists)
df = pd.concat([df,df1],axis=1)
#print("At The Money : " , round(price,-2))


#Calculating the total premium paid or recieved 

filter_col = [col for col in list(df) if col.startswith(('PE_','CE_'))]
df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
for d in df[filter_col]:
    df['Total_Prem'] = df[filter_col].sum(axis=1)   # Total_PP = total put premium 


#Calculating the total Profit and Loss

df['Total_P&L'] = df['Total_Prem'][0] - df['Total_Prem']

#print('Profit at expiry period',df[-1:]['Total_P&L'])


{'Current_Price': 17837, 'Annual_Drift': 0.07, 'Annual_Volatiltiy': 0.15, 'Time_Period': 0.003968254, 'rate_return': 0.07, 'sigma': 0.1332, 'No_of_days': 30}


In [4]:
#calculating greeks for call option

datalist2 = dict()

K   =   [int(round(price, -2))+800,int(round(price, -2))+500]

for k in K:
    call_theta  = []
    call_gamma  = []
    call_delta  = []
    call_rho    = []
    call_vega   = [] 
    for last_price,T in zip(df['Price'],range(30,0,-1)):
        bsm=op.black_scholes(K=k, St=last_price, r=7, t=T,v=13.32, type='c')   
        call_theta.append(bsm['greeks']['theta'])
        call_gamma.append(bsm['greeks']['gamma'])
        call_delta.append(bsm['greeks']['delta'])
        call_rho.append(bsm['greeks']['rho'])
        call_vega.append(bsm['greeks']['vega'])

        datalist2['theta_'+str(k)]   = call_theta
        datalist2['gamma_'+str(k)]   = call_gamma
        datalist2['delta_'+str(k)]   = call_delta
        datalist2['rho_'+str(k)]     = call_rho
        datalist2['vega_'+str(k)]    = call_vega

df2 =   pd.DataFrame(datalist2)

#new = pd.DataFrame.from_dict(df2)

#Calculating the greeks for put options

datalist3 = dict()

K   =   [int(round(price, -2))-800,int(round(price, -2))-500]

for k in K:
    put_theta   = []
    put_gamma   = []
    put_delta   = []
    put_rho     = []
    put_vega    = [] 
    for last_price,T in zip(df['Price'],range(30,0,-1)):
        bsm     = op.black_scholes(K=k, St=last_price, r=7, t=T,v=13.32, type='p')   
        put_theta.append(bsm['greeks']['theta'])
        put_gamma.append(bsm['greeks']['gamma'])
        put_delta.append(bsm['greeks']['delta'])
        put_rho.append(bsm['greeks']['rho'])
        put_vega.append(bsm['greeks']['vega'])

        datalist3['theta_'+str(k)]  = put_theta
        datalist3['gamma_'+str(k)]  = put_gamma
        datalist3['delta_'+str(k)]  = put_delta
        datalist3['rho_'+str(k)]    = put_rho
        datalist3['vega_'+str(k)]   = put_vega
df3 = pd.DataFrame(datalist3)

greeks_df = pd.concat([df2,df3],axis = 1)

In [5]:
#Changing the signs of greeks according to options selling and buying

put_sell = ['theta_','gamma_','delta_','rho_','vega_']
for data in put_sell:
    greeks_df[data+str((round(price, -2))-500)] = - greeks_df[data+str((round(price, -2))-500)]

call_sell = ['theta_','gamma_','delta_','rho_','vega_']
for data in call_sell:
    greeks_df[data+str((round(price, -2))+500)] = - greeks_df[data+str((round(price, -2))+500)]

In [6]:
#Calculating greeks for overall data

list1 = ['theta','gamma','delta','rho','vega']

datalist4 = dict()

for data in list1:
    filter_col = [col for col in list(greeks_df) if col.startswith(data)]
    greeks_df['Port_'+ data] = 0 # initialize value, otherwise throws KeyErrorx
    for d in greeks_df[filter_col]:
        greeks_df['Port_'+ data] = greeks_df[filter_col].mean(axis=1)    

final_df = pd.concat([df,greeks_df],axis = 1)

In [7]:
final_df.to_csv('Greeks.csv')

In [8]:
final_df

,Count,Price,Change_price,tenor,CE_18600,PE_17000,CE_18300,PE_17300,Total_Prem,Total_P&L,...,theta_17300,gamma_17300,delta_17300,rho_17300,vega_17300,Port_theta,Port_gamma,Port_delta,Port_rho,Port_vega
0,1,17837.000000,28.248639,0.083333,-65.420445,-2.438943e+01,133.872154,6.203663e+01,106.098903,0.000000,...,2.249227e+00,-3.658075e-04,1.659621e-01,2.483316e+00,-1.274174e+01,0.591451,-0.000073,-0.010445,-0.131531,-2.525946
1,2,17865.248639,20.692478,0.080556,-66.892674,-2.096737e+01,137.604846,5.525470e+01,104.999511,1.099392,...,2.198379e+00,-3.523656e-04,1.531008e-01,2.216359e+00,-1.190203e+01,0.607592,-0.000073,-0.012685,-0.159397,-2.478138
2,3,17885.941117,50.392449,0.077778,-66.958887,-1.832868e+01,138.972485,4.996612e+01,103.651039,2.447865,...,2.160832e+00,-3.422223e-04,1.430184e-01,2.000002e+00,-1.118668e+01,0.624567,-0.000075,-0.014469,-0.178830,-2.437848
3,4,17936.333566,21.251118,0.075000,-72.797599,-1.429367e+01,150.447649,4.108414e+01,104.440523,1.658380,...,2.022693e+00,-3.139677e-04,1.234056e-01,1.667173e+00,-9.952390e+00,0.629255,-0.000073,-0.018553,-0.227024,-2.312419
4,5,17957.584684,154.034246,0.072222,-72.966079,-1.220689e+01,152.194432,3.652269e+01,103.544161,2.554742,...,1.964586e+00,-3.014716e-04,1.136351e-01,1.479098e+00,-9.224162e+00,0.642664,-0.000074,-0.020592,-0.245137,-2.251247
5,6,18111.618929,-383.622453,0.069444,-104.991545,-6.069100e+00,207.588916,2.058225e+01,117.110520,-11.011617,...,1.459982e+00,-2.144367e-04,7.076129e-02,8.915589e-01,-6.417488e+00,0.550855,-0.000056,-0.031413,-0.369759,-1.679140
6,7,17727.996476,81.285771,0.066667,-30.833411,-2.410051e+01,75.912891,6.651869e+01,87.497662,18.601241,...,2.786564e+00,-4.523050e-04,1.928776e-01,2.291458e+00,-1.245011e+01,0.733964,-0.000096,-0.001095,0.001460,-2.646448
7,8,17809.282247,108.348001,0.063889,-37.399080,-1.658308e+01,90.445721,4.963131e+01,86.094875,20.004028,...,2.543316e+00,-3.997017e-04,1.547314e-01,1.767204e+00,-1.064065e+01,0.771178,-0.000097,-0.008853,-0.085941,-2.572899
8,9,17917.630248,183.913849,0.061111,-49.479842,-9.827992e+00,115.604184,3.277075e+01,89.067099,17.031804,...,2.140420e+00,-3.242540e-04,1.116005e-01,1.224607e+00,-8.357579e+00,0.773408,-0.000091,-0.019344,-0.195632,-2.349183
9,10,18101.544097,66.690502,0.058333,-81.704903,-3.864610e+00,176.382958,1.537382e+01,106.187267,-0.088363,...,1.418888e+00,-2.044347e-04,5.942831e-02,6.275278e-01,-5.133536e+00,0.651692,-0.000067,-0.034936,-0.348684,-1.688401
